In [ ]:
!nvidia-smi -L

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip -qq '/content/drive/MyDrive/isic-2019.zip'

In [ ]:
# ==== Install Dependencies

!pip install -q efficientnet-pytorch
!pip install -q albumentations
!pip install -q pytorch_ranger

In [ ]:
# ==== Import Libraries

import pandas as pd
import numpy as np

import torch
import torch.nn.functional as F
import torch.nn as nn
import seaborn as sns
import random
import os


import albumentations as aug
from albumentations.pytorch.transforms import ToTensor
import matplotlib.pyplot as plt


from tqdm import tqdm

from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, roc_auc_score
from torch.utils.data import Dataset,DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

import warnings
warnings.filterwarnings("ignore") 
warnings.filterwarnings("ignore", category=DeprecationWarning) 

from pytorch_ranger import Ranger

from efficientnet_pytorch import EfficientNet
from pathlib import Path

from torchvision import transforms

from sklearn.model_selection import train_test_split as tts

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(42)

#Data Loader and Model optimizers

In [ ]:
dataset    = Cloader(image_path,targets,resize=None,transforms=None)
""" returns {'image':tensor_image,'targets':tensor_labels} """
dataloader = DataLoader(dataset,batch_size=64
                                     ,shuffle=True,num_workers=4)
                                     
""" using next(iter(dataloader)) returns a dictionary with keys 'image'
 and 'targets'."""

In [ ]:
es = EarlyStop(patience=7, mode="max", delta=0.0001) 

""" Sample code """
for epoch in range(epochs):
    epoch_score = Trainer.evaluate(......)
    es(epoch_score , model , model_path ="./best.pth")
    """ model_path is the location+filename to save the best model """
    if es.early_stop=True:
	    break

es.reset()  """ resets the best_epoch_score, if in case training multiple
                folds without creating 'es' object again and again."""

In [ ]:
trainer        = Trainer(model,optimizer,device,train_scheduler = None,
                 val_scheduler = None,accumulation_steps=1,fp16=False,
                use_mean_loss=False,checkpoint=None,save_path = "./")
                
""" Training and Evaluating """
train_loss              = trainer.train(train_dataloader)
y_true ,y_pred,val_loss = trainer.evaluate(val_dataloader)

""" Prediction """
y_preds                 = trainer.predict(test_dataloader)  

""" In depths """
""" train_scheduler/val_scheduler : call scheduler.step() while training
                                    or after validating
    accumulation_step             : implements gradient accumulation 
                                    (default = 1)
    fp16                          : mixed precision training
    use_mean_loss                 : loss.mean().backward()
                                    (false if loss is already meaned)
    checkpoint                    : torch.load(checkpoint),loads the 
                                    checkpoint and resumes training.
    save_path                     : location to save the last epoch 
                                    weights                             """

""" Having problem with non resumable training epochs? """    
trainer.saver()  """ saves the last epoch to the save_path location """

""" dataloader must be set in the same state to resume training 
Use [https://gist.github.com/usamec/1b3b4dcbafad2d58faa71a9633eea6a5]
an implementation of ResumableRandomSampler()  """

sampler = ResumableRandomSampler(dataset)
loader  = DataLoader(dataset, batch_size=64, sampler=sampler)
torch.save(sampler.get_state(), "test_samp.pth") -- save the state
sampler.set_state(torch.load("test_samp.pth"))  -- load and set the state

""" and tadaa , resume training 100 % :) """

In [ ]:
logger   = Logger(path="./")  """ save path for logger"""
logger.write( message ,verbose = 1) """ verbose to print the message"""

""" Helps Keep Track of Training """

In [ ]:
saver    = Saver(path="./" , mode = "max")
""" saves the model, optimizer and scheduler based on score and mode """

saver.save(model,optimizer,scheduler,metric)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.base_model = timm.create_model('resnet18',pretrained=True,num_classes=1)
    def forward(self, image, targets):
        batch_size, _, _, _ = image.shape
        out = self.base_model(image)
        loss = nn.BCEWithLogitsLoss()(out.view(batch_size,), targets.type_as(out))
        return out, loss

# Data Exploration

In [ ]:
df = pd.read_csv('/content/ISIC_2019_Training_GroundTruth.csv')
df['label'] = np.argmax(np.array(df.drop(['image'],axis=1)),axis=1)
classes = ['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC'] 

df.head(5)

In [ ]:
np.sum(df.label.values==4)

In [ ]:
wb = np.array([4522,12875,3323,867,2624,239,253,628])

In [ ]:
p = (wb.sum()-wb)/wb.sum()

In [ ]:
def softmax(array):
    return np.exp(array)/np.sum(np.exp(array),axis=1).reshape(-1,1)

In [ ]:
np.exp(p)/(np.sum(np.exp(p)))

In [ ]:
# print(df['MEL'].value_counts())
# print(df['NV'].value_counts())
# print(df['BCC'].value_counts())
# print(df['AK'].value_counts())
# print(df['BKL'].value_counts())
# print(df['DF'].value_counts())
# print(df['VASC'].value_counts())
# print(df['SCC'].value_counts())

In [ ]:
save_root = "/content/drive/MyDrive/ACMCancer/"

In [ ]:
training_data_path = "/content/ISIC_2019_Training_Input/ISIC_2019_Training_Input"

# X_train , X_val ,Y_train , Y_val = tts(df, df.label.values, test_size=0.20
#                                        ,random_state=42,stratify=df.label.values)
# X_train       = X_train.reset_index(drop=True)
# X_val         = X_val.reset_index(drop=True)

X_train = pd.read_csv('/content/drive/MyDrive/ACMCancer/train.csv')
X_val   = pd.read_csv('/content/drive/MyDrive/ACMCancer/val.csv')

In [ ]:
# ===== Augmentations

image_size=224
transforms_train = aug.Compose([
    aug.Resize(image_size, image_size),
    # aug.Transpose(p=0.5),
    aug.VerticalFlip(p=0.5),
    aug.HorizontalFlip(p=0.5),
    aug.RandomBrightness(limit=0.2, p=0.75),
    aug.RandomContrast(limit=0.2, p=0.75),
    # aug.OneOf([
    #     aug.MotionBlur(blur_limit=5),
    #     aug.MedianBlur(blur_limit=5),
    #     # aug.GaussianBlur(blur_limit=5),
    #     aug.GaussNoise(var_limit=(5.0, 30.0)),
    # ], p=0.7),

    aug.OneOf([
        aug.OpticalDistortion(distort_limit=1.0),
        aug.GridDistortion(num_steps=5, distort_limit=1.),
        aug.ElasticTransform(alpha=3),
    ], p=0.7),

    aug.CLAHE(clip_limit=4.0, p=0.7),
    aug.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5),
    aug.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85),
    aug.Cutout(max_h_size=int(image_size * 0.375), max_w_size=int(image_size * 0.375), num_holes=1, p=0.5),    
    aug.Normalize()
])

transforms_val = aug.Compose([
    aug.Resize(image_size, image_size),
    aug.Normalize()
])

In [ ]:
# !pip install -U git+https://github.com/ildoonet/cutmix
# from cutmix.cutmix import CutMix
# from cutmix.utils import CutMixCrossEntropyLoss

In [ ]:
train_images     = X_train.image.values.tolist()
train_images     = [os.path.join(training_data_path, i+".jpg") for i in train_images]

test_images      = X_val.image.values.tolist()
test_images      = [os.path.join(training_data_path, i+".jpg") for i in test_images]

train_dataset    = Cloader(train_images,X_train.label.values,None,transforms_train)
#train_dataset    = CutMix(train_dataset, num_class=5, beta=1.0, prob=0.5, num_mix=3)
test_dataset     = Cloader(test_images,X_val.label.values,None,transforms_val)

train_dataloader = DataLoader(train_dataset,batch_size=32,shuffle=True,num_workers=2)
val_dataloader   = DataLoader(test_dataset,batch_size=32,shuffle=False,num_workers=2)

device           = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# ===== Define model

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.base_model = EfficientNet.from_pretrained('efficientnet-b0',num_classes=8)
    def forward(self, image, targets):
        batch_size, _, _, _ = image.shape
        out = self.base_model(image)
        targets = torch.tensor(targets,dtype=torch.int64)
        weights = torch.tensor([0.11717086, 0.08425763, 0.12285029, 0.13535795, 0.12628751,0.13875566, 0.13867899, 0.13664111])
        loss = nn.CrossEntropyLoss(weight=weights.cuda())(out.view(batch_size,8), targets)
        return out, loss

model = Net()
model.to(device);
model.load_state_dict(torch.load("/content/drive/MyDrive/ACMCancer/best17decagain.pth"))

In [ ]:
optimizer = Ranger(model.parameters(),lr=1e-4)
scheduler = ReduceLROnPlateau(optimizer,factor=0.8, mode="min", patience=2)

trainer   = Trainer(model=model,optimizer=optimizer,device=device,val_scheduler=scheduler)
logger    = Logger()

es        = EarlyStop(patience=5,mode="min") # mode = min to minimise loss

In [ ]:
epochs = 30

for epoch in range(epochs):
    logger.write(f"+ ===== Epoch {epoch+1}/{epochs} ===== +")
    train_loss              = trainer.train(train_dataloader)
    y_true,y_pred ,val_loss = trainer.evaluate(val_dataloader)
    y_pred                  = softmax(y_pred)
    accuracy                = accuracy_score(y_true,np.argmax(y_pred,axis=1))
    es(val_loss,model,model_path ="/content/drive/MyDrive/ACMCancer/best17dec_again.pth")
    logger.write(f"train_loss {train_loss} val_loss {val_loss} ")
    logger.write(f"val accuracy_score {accuracy} ")
    logger.write(" ")
    if es.early_stop:
        break

In [ ]:
es        = EarlyStop(patience=7,mode="min")
optimizer = Ranger(model.parameters(),lr=1e-5)
scheduler = ReduceLROnPlateau(optimizer,factor=0.6, mode="min", patience=2)

trainer   = Trainer(model=model,optimizer=optimizer,device=device,val_scheduler=scheduler)

In [ ]:
epochs = 30

for epoch in range(epochs):
    logger.write(f"+ ===== Epoch {epoch+1}/{epochs} ===== +")
    train_loss              = trainer.train(train_dataloader)
    y_true,y_pred ,val_loss = trainer.evaluate(val_dataloader)
    y_pred                  = softmax(y_pred)
    accuracy                = accuracy_score(y_true,np.argmax(y_pred,axis=1))
    es(val_loss,model,model_path ="/content/drive/MyDrive/ACMCancer/best17decagain.pth")
    logger.write(f"train_loss {train_loss} val_loss {val_loss} ")
    logger.write(f"val accuracy_score {accuracy} ")
    logger.write(" ")
    if es.early_stop:
        break

In [ ]:
epochs = 30

for epoch in range(epochs):
    logger.write(f"+ ===== Epoch {epoch+1}/{epochs} ===== +")
    train_loss              = trainer.train(train_dataloader)
    y_true,y_pred ,val_loss = trainer.evaluate(val_dataloader)
    y_pred                  = softmax(y_pred)
    accuracy                = accuracy_score(y_true,np.argmax(y_pred,axis=1))
    es(val_loss,model,model_path ="/content/drive/MyDrive/ACMCancer/best17decagain2.pth")
    logger.write(f"train_loss {train_loss} val_loss {val_loss} ")
    logger.write(f"val accuracy_score {accuracy} ")
    logger.write(" ")
    if es.early_stop:
        break

In [ ]:
model.load_state_dict(torch.load("/content/drive/MyDrive/ACMCancer/best17decagain.pth"))

In [ ]:
torch.save(model.base_model.cpu(),'/content/drive/MyDrive/ACMCancer/canc.pth')

In [ ]:
model.base_model

In [ ]:
# train_dataset    = Cloader(train_images,X_train.label.values,None,transforms_train)
#train_dataset    = CutMix(train_dataset, num_class=5, beta=1.0, prob=0.5, num_mix=3)


In [ ]:
cls0=['ISIC_0032586', 'ISIC_0058999', 'ISIC_0071132', 'ISIC_0063369',
       'ISIC_0063732', 'ISIC_0070516', 'ISIC_0026811', 'ISIC_0032736',
       'ISIC_0032447']

cls1=['ISIC_0055906', 'ISIC_0058529', 'ISIC_0030688',
       'ISIC_0061304', 'ISIC_0024934', 'ISIC_0056288']

cls2=['ISIC_0056157', 'ISIC_0069185', 'ISIC_0029129', 'ISIC_0058984',
       'ISIC_0068505', 'ISIC_0058310', 'ISIC_0071757', 'ISIC_0072638',
       'ISIC_0058039', 'ISIC_0067819', 'ISIC_0063572']

cls3 = ['ISIC_0067755', 'ISIC_0060814', 'ISIC_0071417', 'ISIC_0071605',
       'ISIC_0070507', 'ISIC_0053688', 'ISIC_0069027', 'ISIC_0064638',
       'ISIC_0059108', 'ISIC_0056643', 'ISIC_0057387']

cls4 = ['ISIC_0031061', 'ISIC_0026276', 'ISIC_0071344',
       'ISIC_0012854_downsampled', 'ISIC_0025297', 'ISIC_0059055',
       'ISIC_0063201', 'ISIC_0053750', 'ISIC_0033758', 'ISIC_0054965',
       'ISIC_0012955_downsampled', 'ISIC_0029102', 'ISIC_0025510']

cls5 = ['ISIC_0060327', 'ISIC_0069653', 'ISIC_0025594', 'ISIC_0053519',
       'ISIC_0060024', 'ISIC_0029824', 'ISIC_0061232', 'ISIC_0063831',
       'ISIC_0063605', 'ISIC_0067025', 'ISIC_0025223']

cls6 = ['ISIC_0065784', 'ISIC_0028188', 'ISIC_0027256', 'ISIC_0033844',
       'ISIC_0031996', 'ISIC_0070113', 'ISIC_0025924', 'ISIC_0033230',
       'ISIC_0032057', 'ISIC_0057994', 'ISIC_0026393', 'ISIC_0033969',
       'ISIC_0030722', 'ISIC_0031217', 'ISIC_0055650']

cls7 = ['ISIC_0056522', 'ISIC_0026466', 'ISIC_0033084', 'ISIC_0061988',
       'ISIC_0024843', 'ISIC_0059953', 'ISIC_0030991', 'ISIC_0055107',
       'ISIC_0025577', 'ISIC_0068997', 'ISIC_0063275', 'ISIC_0030821',
       'ISIC_0025831', 'ISIC_0057224', 'ISIC_0068229', 'ISIC_0032173',
       'ISIC_0029067', 'ISIC_0060333', 'ISIC_0029549', 'ISIC_0061451',
       'ISIC_0057610', 'ISIC_0068938', 'ISIC_0029362']

In [ ]:

test_dataset     = Cloader(test_images,0,None,transforms_val)

# train_dataloader = DataLoader(train_dataset,batch_size=32,shuffle=True,num_workers=2)
val_dataloader   = DataLoader(test_dataset,batch_size=1,shuffle=False)

In [ ]:
image = next(iter(val_dataloader))

In [ ]:
from PIL import Image

In [ ]:
img ='ISIC_0063369'
test_images = os.path.join(training_data_path,img +".jpg")
image = Image.open(test_images)
image = np.array(image)
augmented = transforms_val(image=image)
image = augmented["image"]
image = np.transpose(image, (2, 0, 1)).astype(np.float32)
image = torch.tensor([image], dtype=torch.float)

In [ ]:
model.base_model(image)

In [ ]:
torch.save(model.state_dict(),"/content/drive/MyDrive/ACMCancer/last.pth")